In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import isaacgym
import isaacgymenvs
from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, get_rlgames_env_creator

from rl_games.common import env_configurations, vecenv
from rl_games.torch_runner import Runner
from rl_games.algos_torch import model_builder

from omegaconf import DictConfig, OmegaConf

import torch
import numpy as np
import matplotlib.pyplot as plt

from isaacgymenvs.qp.fista import ForceQP, LocationQP, FISTA
from isaacgymenvs.qp.qp_utils import *
from isaacgymenvs.qp.vecrobotics import *

In [ ]:
cfg = OmegaConf.load("../isaacgymenvs/cfg/config.yaml")
cfg.task_name = "TrifingerNYU"
cfg.num_envs = 1
cfg.task = OmegaConf.load("../isaacgymenvs/cfg/task/TrifingerNYU.yaml")

cfg.task.env.command_mode = "torque"
cfg.task.env.reset_distribution.object_initial_state.type = "default"
cfg.task.normalize_action = False
cfg.headless = True

In [ ]:
def create_env_thunk(**kwargs):
    envs = isaacgymenvs.make(
            cfg.seed, 
            cfg.task_name, 
            cfg.task.env.numEnvs, 
            cfg.sim_device,
            cfg.rl_device,
            cfg.graphics_device_id,
            cfg.headless,
            cfg.multi_gpu,
            cfg.capture_video,
            cfg.force_render,
            cfg,
            **kwargs,
        )
    return envs

In [ ]:
lifting_data = np.load("data/lifting2.npz", allow_pickle=True)["data"]

In [ ]:
device = cfg.sim_device
envs = create_env_thunk()

In [ ]:
# get ftip states
envs.reset_idx(torch.arange(cfg.num_envs))
N = 750
action_buffer = torch.zeros(N, 9).to(device)
ftip_pos_buffer = torch.zeros(N, 3, 3).to(device)
ftip_pos_local_buffer = torch.zeros(N, 3, 3).to(device)
object_pose_buffer = torch.zeros(N, 7).to(device)
contact_normals_buffer = []

q_buffer = torch.zeros(N, 9).to(device)
dq_buffer = torch.zeros(N, 9).to(device)
obs, rwds, resets, info = envs.step(torch.zeros(cfg.num_envs, 9))

In [ ]:
# construct force QP
batch_size = cfg.num_envs
num_vars = 9
lb = -10 * torch.ones(batch_size, num_vars)
ub = 10 * torch.ones(batch_size, num_vars)
mg = torch.tensor([0, 0, 9.81]).repeat(batch_size, 1).to(device)
cost_weights = [1, 200, 1e-4]

for n in range(N):
    q = envs._dof_position
    dq = envs._dof_velocity
    q_buffer[n] = q[0]
    dq_buffer[n] = dq[0]
    
    ftip_state = envs._rigid_body_state[:, envs._fingertip_indices]
    ftip_pos = ftip_state[:, :, 0:3]
    ftip_vel = ftip_state[:, :, 7:10]
    ftip_pos_buffer[n] = ftip_pos[0]
    
    object_pose = envs._object_state_history[0][:, 0:7]
    object_orn = quat2mat(object_pose[:, 3:])
    object_pose_buffer[n] = object_pose[0]
    
    time_idx = 0 if n < 200 else 20 * (n - 200)
    ftip_pos_des = torch.tensor(lifting_data[time_idx]['policy']['controller']['ft_pos_des'], dtype=torch.float32).to(device)
    ftip_vel_des = torch.tensor(lifting_data[time_idx]['policy']['controller']['ft_vel_des'], dtype=torch.float32).to(device)

    ftip_pos_diff = ftip_pos_des - ftip_pos.reshape(cfg.num_envs, 9)
    ftip_vel_diff = ftip_vel_des - ftip_vel.reshape(cfg.num_envs, 9)
    
    task_space_force = torch.tensor([200,200,200]*3, dtype=torch.float32, device=envs.device) * ftip_pos_diff
    task_space_force += torch.tensor([5]*9, dtype=torch.float32, device=envs.device)  * ftip_vel_diff
    jacobian = envs.get_fingertip_jacobian_linear()
    jacobian_transpose = torch.transpose(jacobian, 1, 2)
    
    action = bmv(jacobian_transpose, task_space_force)
    
    action_buffer[n] = action[0]
    obs, rwds, resets, info = envs.step(action)

In [20]:
weights = [1, 10]
batch_size, num_ftip, _ = ftip_pos.shape
num_vars = num_ftip * 3
diag_idx = torch.arange(num_vars)

Q1 = torch.zeros(batch_size, num_vars, num_vars)
Q1[:, diag_idx, diag_idx] = 1.
q1 = -2 * torch.zeros_like(action)

Q2 = torch.zeros(batch_size, num_vars, num_vars)
Q2[:, diag_idx, diag_idx] = 1.
q2 = -2 * bmv(jacobian_transpose, task_space_force)


w1, w2 = weights
Q = w1*Q1 + w2*Q2
q = w1*q1 + w2*q2

In [21]:
lb = -10 * torch.ones(envs.num_envs, num_vars)
ub = 10 * torch.ones(envs.num_envs, num_vars)

prob = LocationQP(batch_size, num_vars, device=device)
qp_solver = FISTA(prob, device=device)
max_it = 20

qp_solver.prob.set_data(Q, q, lb, ub)
qp_solver.reset()
for i in range(max_it):
    qp_solver.step()



In [22]:
qp_solver.prob.yk

tensor([[ 3.4493,  1.1705,  2.9611,  2.1371, -0.3114,  0.7941,  2.1605,  0.2112,
          0.9572]], device='cuda:0')

In [23]:
action

tensor([[ 3.7942,  1.2876,  3.2573,  2.3508, -0.3425,  0.8735,  2.3765,  0.2323,
          1.0529]], device='cuda:0')

In [24]:
bmv(jacobian_transpose, task_space_force)

tensor([[ 3.7942,  1.2876,  3.2573,  2.3508, -0.3425,  0.8735,  2.3765,  0.2323,
          1.0529]], device='cuda:0')